a. import the package needed & define the deivce used

In [10]:
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from torch.nn.utils.rnn import pad_sequence
import torch.nn as nn
from torch.nn import functional as F
import math
device = 'cuda' if torch.cuda.is_available() else 'cpu'

b. mounting the drive

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


c.open file & train the sentence piece encoder

In [5]:
with open("/content/drive/MyDrive/files/dataset.txt" , "r" , encoding="utf-8") as file:
  text = file.read()

import sentencepiece as spm
spm.SentencePieceTrainer.train(input='/content/drive/MyDrive/files/dataset.txt', model_prefix='m', vocab_size=3000) #setting vocab to higher value to encode sub words
sp = spm.SentencePieceProcessor(model_file='/content/m.model')

d. Load the dataset and encode it

In [6]:
df = pd.read_csv("/content/drive/MyDrive/files/dataset.csv")
df.columns=["question" , "answer"]

In [7]:
def encode_data():
  x = [torch.tensor(sp.encode(i)) for i in df["question"]]
  y = [torch.tensor(sp.encode(i)) for i in df["answer"]]
  padding_x  = pad_sequence(x , batch_first=True , padding_value=0)
  padding_y  = pad_sequence(y , batch_first=True , padding_value=0)

  desired_padding = padding_x.size(1) - padding_y.size(1)
  padding_y =  F.pad(padding_y , (0, desired_padding), value=0)
  return padding_x , padding_y

X,Y = encode_data()

Creating the decoders

In [8]:
def scaled_dot_product(q, k, v, mask=None):
    # q: 30 x 8 x 200 x 64, k: 30 x 8 x 200 x 64, v: 30 x 8 x 200 x 64, mask 200 x 200
    d_k = q.size()[-1]
    scaled = torch.matmul(q, k.transpose(-1, -2)) / math.sqrt(d_k) # 30 x 8 x 200 x 200
    print(f"scaled.size() : {scaled.size()}")
    if mask is not None:
        print(f"-- ADDING MASK of shape {mask.size()} --")
        scaled += mask # 30 x 8 x 200 x 200
    attention = F.softmax(scaled, dim=-1) # 30 x 8 x 200 x 200
    values = torch.matmul(attention, v) # 30 x 8 x 200 x 64
    return values, attention


class PositionwiseFeedForward(nn.Module):
    def __init__(self, d_model, hidden, drop_prob=0.1):
        super(PositionwiseFeedForward, self).__init__()
        self.linear1 = nn.Linear(d_model, hidden)
        self.linear2 = nn.Linear(hidden, d_model)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=drop_prob)

    def forward(self, x):
        #  x: 30 x 200 x 512
        x = self.linear1(x) #30 x 200 x 2048
        print(f"x after first linear layer: {x.size()}")
        x = self.relu(x) #30 x 200 x 2048
        print(f"x after relu layer: {x.size()}")
        x = self.dropout(x) #30 x 200 x 2048
        print(f"x after dropout layer: {x.size()}")
        x = self.linear2(x) #30 x 200 x 512
        print(f"x after 2nd linear layer: {x.size()}")
        return x #30 x 200 x 512


class LayerNormalization(nn.Module):
    def __init__(self, parameters_shape, eps=1e-5):
        super().__init__()
        self.parameters_shape=parameters_shape
        self.eps=eps
        self.gamma = nn.Parameter(torch.ones(parameters_shape)) # 512
        self.beta =  nn.Parameter(torch.zeros(parameters_shape)) # 512

    def forward(self, inputs):
        # inputs : 30 x 200 x 512
        dims = [-(i + 1) for i in range(len(self.parameters_shape))] # [-1]
        print(f"dims: {dims}")
        mean = inputs.mean(dim=dims, keepdim=True) #30 x 200 x 1
        print(f"Mean ({mean.size()})")
        var = ((inputs - mean) ** 2).mean(dim=dims, keepdim=True) # 30 x 200 x 512
        std = (var + self.eps).sqrt() # 30 x 200 x 512
        print(f"Standard Deviation  ({std.size()})")
        y = (inputs - mean) / std # 30 x 200 x 512
        print(f"y: {y.size()}")
        out = self.gamma * y  + self.beta  # 30 x 200 x 512
        print(f"out: {out.size()}")
        return out

class MultiHeadCrossAttention(nn.Module):

    def __init__(self, d_model, num_heads):
        super().__init__()
        self.d_model = d_model
        self.num_heads = num_heads
        self.head_dim = d_model // num_heads
        self.kv_layer = nn.Linear(d_model , 2 * d_model) # 1024
        self.q_layer = nn.Linear(d_model , d_model)
        self.linear_layer = nn.Linear(d_model, d_model)

    def forward(self, x, y, mask=None):
        batch_size, sequence_length, d_model = x.size() # 30 x 200 x 512
        print(f"x.size(): {x.size()}")
        kv = self.kv_layer(x) # 30 x 200 x 1024
        print(f"kv.size(): {kv.size()}")
        q = self.q_layer(y) # 30 x 200 x 512
        print(f"q.size(): {q.size()}")
        kv = kv.reshape(batch_size, sequence_length, self.num_heads, 2 * self.head_dim)  # 30 x 200 x 8 x 128
        q = q.reshape(batch_size, sequence_length, self.num_heads, self.head_dim)  # 30 x 200 x 8 x 64
        kv = kv.permute(0, 2, 1, 3) # 30 x 8 x 200 x 128
        q = q.permute(0, 2, 1, 3) # 30 x 8 x 200 x 64
        k, v = kv.chunk(2, dim=-1) # K: 30 x 8 x 200 x 64, v: 30 x 8 x 200 x 64
        values, attention = scaled_dot_product(q, k, v, mask) #  30 x 8 x 200 x 64
        print(f"values: {values.size()}, attention:{attention.size()}")
        values = values.reshape(batch_size, sequence_length, d_model) #  30 x 200 x 512
        out = self.linear_layer(values)  #  30 x 200 x 512
        print(f"out after passing through linear layer: {out.size()}")
        return out  #  30 x 200 x 512

#key diffrence between both is that this one will take the KV from the encoder and the Q from the generated (aka logits)

class MultiHeadAttention(nn.Module):

    def __init__(self, d_model, num_heads):
        super().__init__()
        self.d_model = d_model
        self.num_heads = num_heads
        self.head_dim = d_model // num_heads
        self.qkv_layer = nn.Linear(d_model , 3 * d_model) # 1536
        self.linear_layer = nn.Linear(d_model, d_model)

    def forward(self, x, mask=None):
        batch_size, sequence_length, d_model = x.size() # 30 x 200 x 512
        print(f"x.size(): {x.size()}")
        qkv = self.qkv_layer(x) # 30 x 200 x 1536
        print(f"qkv.size(): {qkv.size()}")
        qkv = qkv.reshape(batch_size, sequence_length, self.num_heads, 3 * self.head_dim) # 30 x 200 x 8 x 192
        print(f"qkv after reshape .size(): {qkv.size()}")
        qkv = qkv.permute(0, 2, 1, 3) # 30 x 8 x 200 x 192
        print(f"qkv after permutation: {qkv.size()}")
        q, k, v = qkv.chunk(3, dim=-1) # q: 30 x 8 x 200 x 64, k: 30 x 8 x 200 x 64, v: 30 x 8 x 200 x 64
        print(f"q: {q.size()}, k:{k.size()}, v:{v.size()}")
        values, attention = scaled_dot_product(q, k, v, mask) # values: 30 x 8 x 200 x 64
        print(f"values: {values.size()}, attention:{attention.size()}")
        values = values.reshape(batch_size, sequence_length, self.num_heads * self.head_dim) # 30 x 200 x 512
        print(f"values after reshaping: {values.size()}")
        out = self.linear_layer(values) # 30 x 200 x 512
        print(f"out after passing through linear layer: {out.size()}")
        return out # 30 x 200 x 512

class DecoderLayer(nn.Module):
  def __init__(self , d_model , ffn_hidden , num_heads , drop_prob):
    super(DecoderLayer, self).__init__()
    self.self_attention = MultiHeadAttention(d_model=d_model, num_heads=num_heads)
    self.norm1 = LayerNormalization(parameters_shape=[d_model])
    self.dropout1 = nn.Dropout(p=drop_prob)
    self.encoder_decoder_attention = MultiHeadCrossAttention(d_model=d_model, num_heads=num_heads)
    self.norm2 = LayerNormalization(parameters_shape=[d_model])
    self.dropout2 = nn.Dropout(p=drop_prob)
    self.ffn = PositionwiseFeedForward(d_model=d_model, hidden=ffn_hidden, drop_prob=drop_prob)
    self.norm3 = LayerNormalization(parameters_shape=[d_model])
    self.dropout3 = nn.Dropout(p=drop_prob)
  def forward(self , x, y , decoder_mask):
    _y = y # expected output
    y = self.self_attention(y,mask=decoder_mask) #masking
    y = self.dropout1(y)
    y = self.norm1(y+ _y) #residual connection

    _y = y
    y = self.encoder_decoder_attention(x,y,mask=None)
    y = self.dropout2(y)
    y= self.norm2(y+_y)

    _y = y
    y = self.ffn(y)
    y = self.dropout3(y)
    y = self.norm3(y+_y)

    return y
class SequentialDecoder(nn.Sequential):
  def forward(self , *inputs):
    x,y,mask = inputs
    for module in self._modules.values(): #calling every decoder layer
      y = module(x,y,mask) #keep fiding th layers with the past generated text
    return y

class Decoder(nn.Module):
  def __init__(self , d_model ,ffn_hidden , num_heads , drop_prob , num_layers=1):
    super().__init__()
    self.layers = SequentialDecoder(*[DecoderLayer(d_model , ffn_hidden , num_heads , drop_prob) for _ in range(num_layers)])
  def forward(self , x , y , mask):
    y = self.layers(x,y,mask)
    return y


e. Variable Initialization

In [11]:
batch_size = 30 #number of inputs processed in the same time
d_model = 512 # size of the embeddings for example each token represented with 64 embedding
num_heads = 8 # number of heads that works in parallel in multi headed
num_layers = 5
drop_prob = 0.1
max_sequance = 200
ffn_hidden = 2048
torch.manual_seed(1337)

# let's go for a basic test
x = torch.randn( (batch_size, max_sequance, d_model) )
y = torch.randn( (batch_size, max_sequance, d_model) )
mask = torch.full([max_sequance , max_sequance] , float("-inf"))
mask = torch.triu(mask , diagonal=1)
decoder = Decoder(d_model, ffn_hidden, num_heads, drop_prob, num_layers)
out = decoder(x,y,mask)
mask

x.size(): torch.Size([30, 200, 512])
qkv.size(): torch.Size([30, 200, 1536])
qkv after reshape .size(): torch.Size([30, 200, 8, 192])
qkv after permutation: torch.Size([30, 8, 200, 192])
q: torch.Size([30, 8, 200, 64]), k:torch.Size([30, 8, 200, 64]), v:torch.Size([30, 8, 200, 64])
scaled.size() : torch.Size([30, 8, 200, 200])
-- ADDING MASK of shape torch.Size([200, 200]) --
values: torch.Size([30, 8, 200, 64]), attention:torch.Size([30, 8, 200, 200])
values after reshaping: torch.Size([30, 200, 512])
out after passing through linear layer: torch.Size([30, 200, 512])
dims: [-1]
Mean (torch.Size([30, 200, 1]))
Standard Deviation  (torch.Size([30, 200, 1]))
y: torch.Size([30, 200, 512])
out: torch.Size([30, 200, 512])
x.size(): torch.Size([30, 200, 512])
kv.size(): torch.Size([30, 200, 1024])
q.size(): torch.Size([30, 200, 512])
scaled.size() : torch.Size([30, 8, 200, 200])
values: torch.Size([30, 8, 200, 64]), attention:torch.Size([30, 8, 200, 200])
out after passing through linear lay

tensor([[0., -inf, -inf,  ..., -inf, -inf, -inf],
        [0., 0., -inf,  ..., -inf, -inf, -inf],
        [0., 0., 0.,  ..., -inf, -inf, -inf],
        ...,
        [0., 0., 0.,  ..., 0., -inf, -inf],
        [0., 0., 0.,  ..., 0., 0., -inf],
        [0., 0., 0.,  ..., 0., 0., 0.]])

f. Generate Batch

In [ ]:
trainX = X[:-100]
validX = X[-100:]
trainY  =Y[:-100]
validY = Y[-100:]
position = 0
def get_batch(split):
  if split == "train":
    x = torch.stack([trainX[i] for i in range(position,position+batch_size)])
    y =  torch.stack([trainY[i] for i in range(position,position+batch_size)])
  else :
    ix = torch.randint(100, (batch_size,))
    x = torch.stack([validX[i] for i in ix])
    y =  torch.stack([validY[i] for i in ix])
  x, y = x.to(device), y.to(device)
  return x , y

get_batch("train")

(tensor([[119,  10,  25,  ...,   0,   0,   0],
         [ 61,  12,   7,  ...,   0,   0,   0],
         [ 25,  10, 561,  ...,   0,   0,   0],
         [ 61,   3,  21,  ...,   0,   0,   0]], device='cuda:0'),
 tensor([[ 481, 1563,  124,  ...,    0,    0,    0],
         [  47,   12,   70,  ...,    0,    0,    0],
         [ 380,  128,    8,  ...,    0,    0,    0],
         [  50,  177,  600,  ...,    0,    0,    0]], device='cuda:0'))

g. Transformer Modules

h. Testing

In [ ]:
context = "hello how you doing ?"
converted = [sp.encode(context)]
converted = torch.tensor(converted , device=device)
converted =  F.pad(converted , (0, 307 - len(converted) ), value=0)
print(converted)
print(sp.decode(m.generate(converted, max_number_tokens=20)[0].tolist()))

tensor([[107,  48, 103, 124,  10, 338,   3,  11,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0